In [ ]:
import os

os.environ["JAVA_HOME"] = "/home/nervuzz/tmp/jdk-11.0.2"
os.environ["PATH"] += f':{os.environ["JAVA_HOME"]}/bin'
os.environ["SPARK_HOME"] = "/home/nervuzz/tmp/spark-3.0.3-bin-hadoop3.2"
os.environ["PATH"] += f':{os.environ["SPARK_HOME"]}/bin'
os.environ["PYTHONPATH"] = f'{os.environ["SPARK_HOME"]}/python/'
os.environ["PYTHONPATH"] += f':{os.environ["SPARK_HOME"]}/python/lib/py4j-0.10.9-src.zip'

# export JAVA_HOME="${HOME}/tmp/jdk-11.0.2"
# export PATH="${JAVA_HOME}/bin:${PATH}"
# export SPARK_HOME="${HOME}/tmp/spark-3.0.3-bin-hadoop3.2"
# export PATH="${SPARK_HOME}/bin:${PATH}"
# export PYTHONPATH="${SPARK_HOME}/python/:$PYTHONPATH"
# export PYTHONPATH="${SPARK_HOME}/python/lib/py4j-0.10.9-src.zip:$PYTHONPATH"

In [13]:
from datetime import datetime

import pandas as pd
import pyspark
from pyspark.sql import types, SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("de_camp") \
    .getOrCreate()

22/03/07 20:59:57 WARN Utils: Your hostname, DELL resolves to a loopback address: 127.0.1.1; using 192.168.93.247 instead (on interface eth0)
22/03/07 20:59:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/07 20:59:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
dataset = "assets/fhvhv_tripdata_2021-02.csv"
!head -n 5 $dataset

hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
HV0003,B02764,2021-02-01 00:10:40,2021-02-01 00:21:09,35,39,
HV0003,B02764,2021-02-01 00:27:23,2021-02-01 00:44:01,39,35,
HV0005,B02510,2021-02-01 00:28:38,2021-02-01 00:38:27,39,91,
HV0005,B02510,2021-02-01 00:43:37,2021-02-01 01:23:20,91,228,


In [10]:
df = spark.read \
    .option("header", "true") \
    .csv(dataset)

In [11]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [4]:
# !head -n 1001 $dataset > assets/head_fhvhv_tripdata_2021_02.csv
dataset_head = "assets/head_fhvhv_tripdata_2021_02.csv"

In [16]:
df_pandas = pd.read_csv(dataset_head)

In [18]:
df.dtypes

[('hvfhs_license_num', 'string'),
 ('dispatching_base_num', 'string'),
 ('pickup_datetime', 'string'),
 ('dropoff_datetime', 'string'),
 ('PULocationID', 'string'),
 ('DOLocationID', 'string'),
 ('SR_Flag', 'string')]

In [19]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [21]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [23]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv(dataset)

In [24]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,TimestampType,true),StructField(dropoff_datetime,TimestampType,true),StructField(PULocationID,IntegerType,true),StructField(DOLocationID,IntegerType,true),StructField(SR_Flag,StringType,true)))

In [26]:
df = df.repartition(24)

In [27]:
df.write.parquet("assets/fhvhv/2021/02/")

22/03/06 22:21:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/06 22:21:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/03/06 22:21:11 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [5]:
df = spark.read.parquet("assets/fhvhv/2021/02/")
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [11]:
# Question 2

!ls -lh assets/fhvhv/2021/02

total 206M
-rw-r--r-- 1 nervuzz nervuzz    0 Mar  6 22:21 _SUCCESS
-rw-r--r-- 1 nervuzz nervuzz 8.6M Mar  6 22:21 part-00000-20878ced-5dd4-4d06-9111-e19666883a04-c000.snappy.parquet
-rw-r--r-- 1 nervuzz nervuzz 8.6M Mar  6 22:21 part-00001-20878ced-5dd4-4d06-9111-e19666883a04-c000.snappy.parquet
-rw-r--r-- 1 nervuzz nervuzz 8.6M Mar  6 22:21 part-00002-20878ced-5dd4-4d06-9111-e19666883a04-c000.snappy.parquet
-rw-r--r-- 1 nervuzz nervuzz 8.6M Mar  6 22:21 part-00003-20878ced-5dd4-4d06-9111-e19666883a04-c000.snappy.parquet
-rw-r--r-- 1 nervuzz nervuzz 8.6M Mar  6 22:21 part-00004-20878ced-5dd4-4d06-9111-e19666883a04-c000.snappy.parquet
-rw-r--r-- 1 nervuzz nervuzz 8.6M Mar  6 22:21 part-00005-20878ced-5dd4-4d06-9111-e19666883a04-c000.snappy.parquet
-rw-r--r-- 1 nervuzz nervuzz 8.6M Mar  6 22:21 part-00006-20878ced-5dd4-4d06-9111-e19666883a04-c000.snappy.parquet
-rw-r--r-- 1 nervuzz nervuzz 8.6M Mar  6 22:21 part-00007-20878ced-5dd4-4d06-9111-e19666883a04-c000.snappy.parquet
-rw-r--r-- 1 

In [31]:
df.summary().show()

+-------+-----------------+--------------------+------------------+-----------------+-------+
|summary|hvfhs_license_num|dispatching_base_num|      PULocationID|     DOLocationID|SR_Flag|
+-------+-----------------+--------------------+------------------+-----------------+-------+
|  count|         11613942|            11613942|          11613942|         11613942|   1346|
|   mean|             null|                null|135.25379642846502|138.8317934599639|    1.0|
| stddev|             null|                null| 76.59301944910268|78.51688968882083|    0.0|
|    min|           HV0003|              B02395|                 1|                1|      1|
|    25%|             null|                null|                70|               72|    1.0|
|    50%|             null|                null|               137|              140|    1.0|
|    75%|             null|                null|               209|              213|    1.0|
|    max|           HV0005|              B03136|            

In [32]:
! wc -l $dataset

11613943 assets/fhvhv_tripdata_2021-02.csv


In [18]:
# Question 3
df \
    .filter(F.to_date(df.pickup_datetime) == '2021-02-15') \
    .select('dispatching_base_num', 'pickup_datetime') \
    .count()

367170

In [19]:
# Question 4
df.registerTempTable("table_fhv")

In [54]:
sql_on_df = spark.sql("""
    SELECT hvfhs_license_num,
    dispatching_base_num,
    PULocationID,
    DOLocationID,
    date_trunc('day', pickup_datetime) as day_of_pickup,
    (dropoff_datetime - pickup_datetime) as trip_duration
    FROM table_fhv
""")

In [51]:
sql_on_df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- day_of_pickup: timestamp (nullable = true)
 |-- trip_duration: interval (nullable = true)



In [66]:
# sql_on_df.sort(sql_on_df.trip_duration.cast(IntegerType()).desc())
# sql_on_df.show()
df2 = df \
    .withColumn('diff_in_sec', F.col("dropoff_datetime").cast("long") - F.col('pickup_datetime').cast("long"))
df2.orderBy('diff_in_sec', ascending=False).show(3)
#     .sort(df.diff_in_sec.desc()) \
#     .show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|diff_in_sec|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+
|           HV0005|              B02510|2021-02-11 13:40:44|2021-02-12 10:39:44|         247|          41|   null|      75540|
|           HV0004|              B02800|2021-02-17 15:54:53|2021-02-18 07:48:34|         242|         254|   null|      57221|
|           HV0004|              B02800|2021-02-20 12:08:15|2021-02-21 00:22:14|         188|          55|   null|      44039|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-----------+
only showing top 3 rows

